# Esercizio: moltiplicazione matrice-vettore

Ispirandoti al kernel `matmul_gpu_shared`, completa il kernel `matvec_gpu_shared` che, dati una matrice $m \times k$  $A$  e un vettore $v$, calcoli il prodotto $w = Av$ sfruttando la memoria shared. Quali sono i dati che vengono letti più volte dalla global memory e che quindi necessitano di essere caricari in shared?

### Import e dati

In [1]:
import numpy as np

import numba
from numba import cuda

import math

from matplotlib import pyplot as plt

In [2]:
m = 2**10
k = 2**10
n = 1

A = np.random.randint(10, size = (m,k), dtype = 'int32')
v = np.random.randint(10, size = (k,n), dtype = 'int32')

### Versione Python

In [3]:
# soluzione test
w_np = A@v

In [4]:
def matvec_py(A,v):
    if (v.shape[0] != A.shape[1]):
        print("Errore: le dimensioni delle matrici non sono compatibili!")
        return

    w = np.zeros((m,1))

    for i in range(A.shape[0]):
        tmp = 0
        for k in range(A.shape[1]):
            tmp = tmp + A[i,k]*v[k,0]
        #endfor
        w[i,0] = tmp
    #endfor

    return w

In [5]:
w_py = matvec_py(A,v)

print('|| w_np - w_py || =', np.linalg.norm(w_np-w_py))

|| w_np - w_py || = 0.0


### Versione Kernel

In [6]:
@cuda.jit()
def matvec_gpu(A,v,w):
    i = cuda.grid(1)

    if i < A.shape[0]:
        tmp = 0
        for k in range(A.shape[1]):
            tmp += A[i, k] * v[k,0]
        #endfor
        w[i,0] = tmp
    #endif
    return

In [7]:
w_gpu = np.zeros((m,n), dtype = np.int64)

threadsperblock = 128
blockspergrid = math.ceil(A.shape[0] / threadsperblock)

matvec_gpu[blockspergrid, threadsperblock](A,v,w_gpu)

print('|| w_np - w_gpu || =', np.linalg.norm(w_np-w_gpu))

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


|| w_np - w_gpu || = 0.0


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


### Versione Shared-memory

In [8]:
TPB = 32

@cuda.jit()
def matvec_gpu_shared(A,v,w):
    # Controllo che le matrici A e B siano compatibili per il prodotto
    k = A.shape[1]
    if (v.shape[0] != k):
        return

    i = cuda.grid(1)
    s_i = cuda.threadIdx.x

    A_sh = cuda.shared.array(shape=(TPB,), dtype=numba.int32)
    v_sh = cuda.shared.array(shape=(TPB,), dtype=numba.int32)
    Ntiles = math.ceil(n/TPB)

    if i < A.shape[0]:
        tmp = 0
        for k in Ntiles:
            # Caricamento dati in shared memory
            if (s_i + k * TPB < k):
                A_sh[s_i] = A[i, s_i + k * TPB]
                v_sh[s_i] = v[s_i + k * TPB, 0]
            else:
                A_sh[s_i] = 0
                v_sh[s_i] = 0
            #endif
            cuda.syncthreads()

            tmp += A_sh[s_i] * v_sh[s_i]
            cuda.syncthreads()
        #endfor
        w[i,0] = tmp
    #endif
    return

In [9]:
matvec_gpu[blockspergrid, threadsperblock](A,v,w_gpu)

In [10]:
print('|| w_np - w_gpu || =', np.linalg.norm(w_np-w_gpu))

|| w_np - w_gpu || = 0.0
